In [9]:
import os
os.chdir("/content")  # reset working directory


In [10]:
!pip install unidecode eng_to_ipa inflect pypinyin jieba cn2an librosa pyworld gradio ffmpeg-python faster_whisper whisper_timestamped wavmark pydub --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 48.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 22.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 50.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8

In [11]:
# ======================================================
# 1. Clone OpenVoice to Drive
# ======================================================
!rm -rf /content/drive/MyDrive/OpenVoice
!git clone https://github.com/myshell-ai/OpenVoice.git /content/drive/MyDrive/OpenVoice


Cloning into '/content/drive/MyDrive/OpenVoice'...
remote: Enumerating objects: 460, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 460 (delta 11), reused 7 (delta 7), pack-reused 445 (from 2)
Receiving objects: 100% (460/460), 3.85 MiB | 22.05 MiB/s, done.
Resolving deltas: 100% (218/218), done.


In [15]:
# ------------------------
# Download pretrained converter
# ------------------------
!mkdir -p assets/converter_ckpt
!wget -q -O assets/converter_ckpt/config.json https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/converter/config.json
!wget -q -O assets/converter_ckpt/weights.pth https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/converter/checkpoint.pth

# ------------------------
# Import path fix
# ------------------------
import sys
sys.path.append("/content/drive/MyDrive/OpenVoice/OpenVoice")

# ------------------------
# Mount Drive
# ------------------------
from google.colab import drive
drive.mount('/content/drive')

# ------------------------
# File paths & folders
# ------------------------
import os
from glob import glob
from pydub import AudioSegment

ref_clip = "/content/drive/MyDrive/voice_samples/myaudio3.mp3"
input_folder = "/content/drive/MyDrive/audiobook"   # folder with parts: part_1.mp3, part_2.mp3, ...
converted_folder = "/content/drive/MyDrive/converted_audio_chunks"
final_output = "/content/drive/MyDrive/converted_audio/final_cloned_audiobook.wav"

os.makedirs(converted_folder, exist_ok=True)
os.makedirs(os.path.dirname(final_output), exist_ok=True)

# ------------------------
# Load model on GPU (FP16 if supported)
# ------------------------
import torch
from openvoice.api import ToneColorConverter
from openvoice import se_extractor

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Using device: {device} ({torch.cuda.get_device_name(0) if device=='cuda' else 'CPU'})")

converter = ToneColorConverter("assets/converter_ckpt/config.json", device=device)
converter.load_ckpt("assets/converter_ckpt/weights.pth")

# Extract speaker embedding once (send to GPU if available)
target_se, _ = se_extractor.get_se(ref_clip, converter)
if device == "cuda":
    target_se = target_se.half().to(device)

# ------------------------
# Convert each part (skip if already converted)
# ------------------------
part_files = sorted(glob(os.path.join(input_folder, "*.mp3")))  # sorted to maintain order

for i, part_file in enumerate(part_files):
    output_path = os.path.join(converted_folder, f"chunk_{i}.wav")

    # ✅ Skip already processed files
    if os.path.exists(output_path):
        print(f"⏩ Skipping {part_file}, already converted.")
        continue

    print(f"🎧 Converting {part_file} -> {output_path}")
    with torch.cuda.amp.autocast(enabled=(device=="cuda")):  # FP16 acceleration
        converter.convert(
            audio_src_path=part_file,
            src_se=None,
            tgt_se=target_se,
            output_path=output_path
        )

    # Clear GPU memory after each chunk
    if device == "cuda":
        torch.cuda.empty_cache()

# ------------------------
# Merge all converted chunks
# ------------------------
final_audio = AudioSegment.empty()
converted_parts = sorted(glob(os.path.join(converted_folder, "*.wav")))

for chunk_file in converted_parts:
    final_audio += AudioSegment.from_wav(chunk_file)

final_audio.export(final_output, format="wav")
print(f"🎉 Done! Final audiobook saved at: {final_output}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.

🚀 Using device: cuda (Tesla T4)


/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


step59000_snr39.99_pesq4.35_BERP_none0.3(…):   0%|          | 0.00/10.0M [00:00<?, ?B/s]

...

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip
[(0.238, 5.8578125)]
after vad: dur = 5.619


/usr/local/lib/python3.12/dist-packages/torch/functional.py:730: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /pytorch/aten/src/ATen/native/SpectralOps.cpp:875.)
  return _VF.stft(  # type: ignore[attr-defined]


⏩ Skipping /content/drive/MyDrive/audiobook/part_1.mp3, already converted.
⏩ Skipping /content/drive/MyDrive/audiobook/part_10.mp3, already converted.
⏩ Skipping /content/drive/MyDrive/audiobook/part_100.mp3, already converted.
⏩ Skipping /content/drive/MyDrive/audiobook/part_101.mp3, already converted.
⏩ Skipping /content/drive/MyDrive/audiobook/part_102.mp3, already converted.
⏩ Skipping /content/drive/MyDrive/audiobook/part_103.mp3, already converted.
⏩ Skipping /content/drive/MyDrive/audiobook/part_104.mp3, already converted.
⏩ Skipping /content/drive/MyDrive/audiobook/part_105.mp3, already converted.
⏩ Skipping /content/drive/MyDrive/audiobook/part_106.mp3, already converted.
⏩ Skipping /content/drive/MyDrive/audiobook/part_107.mp3, already converted.
⏩ Skipping /content/drive/MyDrive/audiobook/part_108.mp3, already converted.
⏩ Skipping /content/drive/MyDrive/audiobook/part_109.mp3, already converted.
⏩ Skipping /content/drive/MyDrive/audiobook/part_11.mp3, already converted.
⏩ S

/tmp/ipython-input-4238020376.py:67: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):  # FP16 acceleration


🎧 Converting /content/drive/MyDrive/audiobook/part_146.mp3 -> /content/drive/MyDrive/converted_audio_chunks/chunk_52.wav
🎧 Converting /content/drive/MyDrive/audiobook/part_147.mp3 -> /content/drive/MyDrive/converted_audio_chunks/chunk_53.wav
🎧 Converting /content/drive/MyDrive/audiobook/part_148.mp3 -> /content/drive/MyDrive/converted_audio_chunks/chunk_54.wav
🎧 Converting /content/drive/MyDrive/audiobook/part_149.mp3 -> /content/drive/MyDrive/converted_audio_chunks/chunk_55.wav
🎧 Converting /content/drive/MyDrive/audiobook/part_15.mp3 -> /content/drive/MyDrive/converted_audio_chunks/chunk_56.wav
🎧 Converting /content/drive/MyDrive/audiobook/part_150.mp3 -> /content/drive/MyDrive/converted_audio_chunks/chunk_57.wav
🎧 Converting /content/drive/MyDrive/audiobook/part_151.mp3 -> /content/drive/MyDrive/converted_audio_chunks/chunk_58.wav
🎧 Converting /content/drive/MyDrive/audiobook/part_152.mp3 -> /content/drive/MyDrive/converted_audio_chunks/chunk_59.wav
🎧 Converting /content/drive/MyDri